In [1]:
from google.colab import drive
drive.mount('/content/drive')
FILE_LINKS = {'okpd': "/content/drive/MyDrive/okpd_forml.csv",}

Mounted at /content/drive


In [2]:
!pip3 install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 71.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 105.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 26.6 MB/s eta 0:00:00


In [3]:
import torch
import pandas as pd
import numpy as np
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):
        self.labels = [label for label in df['labels']]
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 64, truncation=True,
                                return_tensors="pt") for text in df['name']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y


In [4]:
from torch import nn
from transformers import BertModel

class BertClassifier(nn.Module):

    def __init__(self, num_classes, dropout=0.2):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear1 = nn.Linear(768, 2304)
        self.linear2 = nn.Linear(2304, 2304*8)
        self.linear3 = nn.Linear(2304*8,2304*8)
        self.linear4 = nn.Linear(2304*8,5000)
        self.linear5 = nn.Linear(5000, num_classes)
        self.relu = nn.LeakyReLU(0.2)

    def forward(self, input_id, mask):
        #_ в коде, содержит векторы встраивания всех токенов в последовательности
        #Вторая переменная pooled_output содержит вектор внедрения токена [CLS].
        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        #dropout_output = self.dropout(pooled_output)
        linear_output = self.linear1(pooled_output)
        linear_output = self.relu(linear_output)
        linear_output = self.linear2(linear_output)
        linear_output = self.relu(linear_output)
        linear_output = self.linear3(linear_output)
        linear_output = self.relu(linear_output)
        linear_output = self.linear4(linear_output)
        linear_output = self.relu(linear_output)
        linear_output = self.linear5(linear_output)
        final_layer = self.relu(linear_output)

        return final_layer

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        mask = X['attention_mask'].to(device)
        X = X['input_ids'].squeeze(1).to(device)
        y = y.to(device)

        # Compute prediction error
        pred = model(X, mask)
        loss = loss_fn(pred, y.long())
    
        # Backpropagation
        model.zero_grad()
        loss.backward()
        optimizer.step()
        # optimizer.zero_grad()
        
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            mask = X['attention_mask'].to(device)
            X = X['input_ids'].squeeze(1).to(device)
            y = y.to(device)
            pred = model(X, mask)
            test_loss += loss_fn(pred, y.long()).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

Using cuda device


In [6]:
from torch.optim import Adam, SGD
#Задаём модель и данные
model = BertClassifier(num_classes = 264)
# for name, param in model.named_parameters():    
#   print(name, param.requires_grad)
# Замораживаем все слои кроме классификатора
# for name, param in model.named_parameters():    
#     if name not in ["linear1.weight", "linear1.bias","linear2.weight", "linear2.bias", "linear3.weight", "linear3.bias", "linear4.weight", "linear4.bias","linear5.weight", "linear5.bias","linear6.weight", "linear6.bias" ]:
#         param.requires_grad = False

#Вводим данные и преобразуем их
df = pd.read_csv(FILE_LINKS['okpd'], sep=',')
df = df.dropna(subset=['name'])
df_train, df_test = np.split(df.sample(frac=1, random_state=42), 
                                     [int(.8*len(df))])
print(df_train.shape[0], df_test.shape[0])

train_data, test_data = Dataset(df_train), Dataset(df_test)
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=4, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=4)

#Задаём функции потерь и оптимизатор
loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr= 0.000003)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


12256 3065


In [7]:
torch.cuda.empty_cache()

In [8]:
epochs = 10
model = model.to(device)
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 5.576480  [    4/12256]
loss: 5.443093  [  404/12256]
loss: 4.599652  [  804/12256]
loss: 5.606054  [ 1204/12256]
loss: 4.390491  [ 1604/12256]
loss: 5.701580  [ 2004/12256]
loss: 4.228402  [ 2404/12256]
loss: 5.107403  [ 2804/12256]
loss: 5.555834  [ 3204/12256]
loss: 5.459588  [ 3604/12256]
loss: 4.202342  [ 4004/12256]
loss: 4.874112  [ 4404/12256]
loss: 5.046617  [ 4804/12256]
loss: 5.358889  [ 5204/12256]
loss: 5.366324  [ 5604/12256]
loss: 3.659627  [ 6004/12256]
loss: 4.260679  [ 6404/12256]
loss: 4.397946  [ 6804/12256]
loss: 4.404627  [ 7204/12256]
loss: 4.574920  [ 7604/12256]
loss: 3.432749  [ 8004/12256]
loss: 4.751901  [ 8404/12256]
loss: 3.136210  [ 8804/12256]
loss: 5.040502  [ 9204/12256]
loss: 4.371366  [ 9604/12256]
loss: 4.801415  [10004/12256]
loss: 5.310869  [10404/12256]
loss: 4.557209  [10804/12256]
loss: 3.871802  [11204/12256]
loss: 4.833205  [11604/12256]
loss: 3.345905  [12004/12256]
Test Error: 
 Accuracy: 9.0%, 

In [9]:
torch.save(model, './Bert_model_5_1.pth')

#Далее то что пойдёт в конечный продукт АНТИГОТОВО

In [ ]:
model = torch.load("./Bert_model_5.pth")

def predict(X)
    model.eval()
    with torch.no_grad():
        pred = []
        torch.utils.data.DataLoader(X, batch_size=1)
        for X, y in dataloader:
            mask = X['attention_mask'].to(device)
            X = X['input_ids'].squeeze(1).to(device)
            y = y.to(device)
            pred += [model(X, mask)]
        
    return pred


In [ ]:
print("DONE!!!!!!")
print("Finally!!")